# Possible Data Preparation Steps

In [1]:
import os
import sys



In [2]:
os.chdir('/Users/emilyli/workspace/datasets/')
# df = pd.read_csv('customer_segmentation.csv')



# Handling Missing Values

# Scaling / Normalization

# Outlier Detection & Removal

# Feature Selection: PCA etc